In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sys
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [3]:
# Funciones

# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
def EncoderViewMinable(df):
    new_dataset = df
    norm = MinMaxScaler()
    norm = norm.fit(new_dataset.values[:,:])
    valMin = norm.data_min_
    valMax = norm.data_max_
    dataRange = norm.data_range_
    #df_norm = norm.fit_transform(df.values[:,:-1])

    return [valMin, valMax, dataRange]




'''
Funcion para generar el vector de pesos aleatorio.
Entradas:
w: Vector de pesos w [0,1], igual al numero de caracteristicas normalizadas.
nc: Nunero de ceros que debe contener el vector de pesos [10,20,30,40,50]
'''


def GenerateWeightVector(w, nc):
    posceros = np.random.choice(len(w), nc, replace=False)
    w[posceros] = 0
    s  = np.sum(w)
    wf = np.round(w/s, 4)
    return wf




# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado





In [ ]:
# Normalización z


In [52]:
# Prueba | Muestra dataset Iris
d = {'A': [5.1,0.9,4.7,7,6.4,6.9,5.5,6.3,5.8,7.1],
        'B': [3.5,3,3.2,3.2,3.2,3.1,2.3,3.3,2.7,3],
        'C': [1.4,1.4,1.3,4.7,4.5,4.9,4,6,5.1,5.9],
        'D': [0.2,0.2,0.2,1.4,1.5,1.5,1.3,2,1.9,2.1],
        'G': [0,0,0,1,1,1,1,2,2,2]}

df = pd.DataFrame(data=d)
print(df)
# 1. Codificacion 
valMin, valMax, dataRange = EncoderViewMinable(df)
print("Valores minimos: ", valMin)
print("Rangos de Valores: ", dataRange)
print("Longitudes : ", len(valMin), len(valMax), len(dataRange))
#2. Normalizamos los datos
df_normalizado = NormalizeViewMinable(df.values[:, :-1],valMin, dataRange)
print(df_normalizado)



     A    B    C    D  G
0  5.1  3.5  1.4  0.2  0
1  0.9  3.0  1.4  0.2  0
2  4.7  3.2  1.3  0.2  0
3  7.0  3.2  4.7  1.4  1
4  6.4  3.2  4.5  1.5  1
5  6.9  3.1  4.9  1.5  1
6  5.5  2.3  4.0  1.3  1
7  6.3  3.3  6.0  2.0  2
8  5.8  2.7  5.1  1.9  2
9  7.1  3.0  5.9  2.1  2
Valores minimos:  [0.9 2.3 1.3 0.2 0. ]
Rangos de Valores:  [6.2 1.2 4.7 1.9 2. ]
Longitudes :  5 5 5
[[0.67741935 1.         0.0212766  0.        ]
 [0.         0.58333333 0.0212766  0.        ]
 [0.61290323 0.75       0.         0.        ]
 [0.98387097 0.75       0.72340426 0.63157895]
 [0.88709677 0.75       0.68085106 0.68421053]
 [0.96774194 0.66666667 0.76595745 0.68421053]
 [0.74193548 0.         0.57446809 0.57894737]
 [0.87096774 0.83333333 1.         0.94736842]
 [0.79032258 0.33333333 0.80851064 0.89473684]
 [1.         0.58333333 0.9787234  1.        ]]


In [6]:
# 3 Generar vector de pesos
np.random.seed(0)
vp = np.random.rand(4)
print("Vector de pesos Original: ", vp)
wi = GenerateWeightVector(vp, 0)
print(wi)
print("Suma vector de Pesos: ", sum(wi))

Vector de pesos Original:  [0.5488135  0.71518937 0.60276338 0.54488318]
[0.2276 0.2966 0.2499 0.2259]
Suma vector de Pesos:  1.0


In [46]:
# Funcion de calidad con voto inverso
def qualityFunction3(df_norm, wi,df,k):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        vrf = df_norm[i]
        print(f"Vector {i} :", vrf) 
        ListaPesosPonderados= [[sys.float_info.max,0] for a in range(k)]
        for j in range(len(df_norm)):
            if i != j:
                print(ListaPesosPonderados)
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("Distancia Euclideana: ", dE)
                if dE < ListaPesosPonderados[k-1][0]:
                    ListaPesosPonderados[k-1][0]=dE
                    ListaPesosPonderados[k-1][1]=j
                    ListaPesosPonderados.sort(key=lambda x: x[0], reverse=False)
        print(f"vector {i} es muy similar a los vectores en la posición {ListaPesosPonderados}")

        dEG0=0
        dEG1=0
        dEG2=0
        for i in range(len(ListaPesosPonderados)):
            index=ListaPesosPonderados[i][1]
            g = int(df.values[index][-1])
            if g==0:
                dEG0= np.power((1/ListaPesosPonderados[i][0]),2) + dEG0
            if g==1:
                dEG1= np.power((1/ListaPesosPonderados[i][0]),2)  + dEG1
            if g==2:
                dEG2= np.power((1/ListaPesosPonderados[i][0]),2)  + dEG2

            
        dis = [dEG0, dEG1,dEG2]
        print("Distancias Inversas Aociadas: ", dis)
        grupoSelected = int(dis.index(max(dis)))
        print("Grupo seleccionado: ", grupoSelected)
        y_pred.append(grupoSelected)
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return [qs, y_pred]


In [22]:
def qualityFunction(df_norm, wi,df,k):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        vrf = df_norm[i]
        print(f"Vector {i} :", vrf) 
        ListaPesosPonderados= [[sys.float_info.max,0] for a in range(k)]
        for j in range(len(df_norm)):
            if i != j:
                print(ListaPesosPonderados)
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("Distancia Euclideana: ", dE)
                if dE < ListaPesosPonderados[k-1][0]:
                    ListaPesosPonderados[k-1][0]=dE
                    ListaPesosPonderados[k-1][1]=j
                    ListaPesosPonderados.sort(key=lambda x: x[0], reverse=False)
        print(f"vector {i} es muy similar a los vectores en la posición {ListaPesosPonderados}")

        grupos = []
        for i in range(len(ListaPesosPonderados)):
            index=ListaPesosPonderados[i][1]
            g = df.values[index][-1]
            grupos.append(g)
        
        print("Grupos asociados: ", grupos)
        grupoSelected = int(pd.Series(grupos).value_counts().index[0])
        print("Grupo seleccionado: ", grupoSelected)
        y_pred.append(grupoSelected)
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return [qs, y_pred]



def qualityFunctionOriginal(df_norm, wi,df):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    posMinDep = 0
    minDep = sys.float_info.max 
    y_pred = []
    for i in range(len(df_norm)):
        vrf = df_norm[i]
        print(f"vercor {i}: {vrf}")
        
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
        #print(posMinDep)
        y_pred.append(df.values[posMinDep][-1])
        print(f"Posición {posMinDep} y distancia {minDep} hasta el momento")
    qs = accuracy_score(df.values[:,-1], y_pred)
    return [qs, y_pred]




In [50]:
# Funcion de calidad
calidad = qualityFunction3(df_normalizado, wi,df,1)


Vector 0 : [0.67741935 1.         0.0212766  0.        ]
[[1.7976931348623157e+308, 0]]
Distancia Euclideana:  0.15593800872933283
[[0.15593800872933283, 1]]
Distancia Euclideana:  0.019597974626315823
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.2532182152739098
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.24301370305636175
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.29647499960550666
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.44973912492244766
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.4588909074788283
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.47044100897094887
[[0.019597974626315823, 2]]
Distancia Euclideana:  0.5301611457494413
vector 0 es muy similar a los vectores en la posición [[0.019597974626315823, 2]]
Distancias Inversas Aociadas:  [2603.6201135545703, 0, 0]
Grupo seleccionado:  0
Vector 1 : [0.         0.58333333 0.0212766  0.        ]
[[1.7976931348623157e+308, 0]]
Distancia Euclideana:  0.15593800872933283
[[0.155938

In [51]:
calidad

[0.9, [0, 0, 0, 1, 1, 1, 2, 2, 2, 2]]

In [24]:
import pandas as pd
gs= [0.0, 0.0, 1.0]
c = pd.Series(gs).value_counts()
c.index[0]

0.0

In [25]:
c

0.0    2
1.0    1
dtype: int64

In [14]:
a = pd.Series(gs).value_counts().index[0]
print(a)

0.0


In [45]:
lista_listas = [[0.156,2],[0.178,1],[0.285, 4]]

grupos = []
dEG0 = 0
dEG1 =0
dEG2=0
for i in range(len(lista_listas)):
    index=lista_listas[i][1]
    g = int(df.values[index][-1])
    print(g)
    
    if g==0:
        dEG0= np.power((1/lista_listas[i][0]),2) + dEG0
    if g==1:
        dEG1= np.power((1/lista_listas[i][0]),2)  + dEG1
    
    if g==2:
        dEG2= np.power((1/lista_listas[i][0]),2)  + dEG2

    grupos.append(g)

dis = [dEG0, dEG1,dEG2]
print("Distancias Inversas Aociadas: ", dis)
final_group = dis.index(max(dis))
print("Grupo Final: ", final_group)

0
Distancias Inversas Aociadas:  [41.091387245233406, 0, 0]
Grupo Final:  0


In [43]:
grupos

[0, 0, 1]